In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
gen_name_gen_id = pd.read_csv("./data/gene_name_gene_id_join_data.txt", sep='\t', header=0)
gen_name_gen_id["gene_id"] = gen_name_gen_id["Gene stable ID"]
gen_name_gen_id["gene_name"] = gen_name_gen_id["Gene name"]
gen_name_gen_id = gen_name_gen_id[["gene_id", "gene_name"]]
gen_name_gen_id = gen_name_gen_id.drop_duplicates(subset=["gene_id"])


Disgenet esetében át kell váltani a gén névről a gén azonosító használatára a kapcsolótábla segítségével.

In [5]:
disgenet = pd.read_csv("./data/disgenet_curated_gene_disease_associations.tsv", sep='\t', header=0)
disgenet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84038 entries, 0 to 84037
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   geneId               84038 non-null  int64  
 1   geneSymbol           84038 non-null  object 
 2   DSI                  83998 non-null  float64
 3   DPI                  83979 non-null  float64
 4   diseaseId            84038 non-null  object 
 5   diseaseName          84038 non-null  object 
 6   diseaseType          84038 non-null  object 
 7   diseaseClass         80401 non-null  object 
 8   diseaseSemanticType  84038 non-null  object 
 9   score                84038 non-null  float64
 10  EI                   79655 non-null  float64
 11  YearInitial          79655 non-null  float64
 12  YearFinal            79655 non-null  float64
 13  NofPmids             84038 non-null  int64  
 14  NofSnps              84038 non-null  int64  
 15  source               84038 non-null 

In [6]:
disgenet_join = pd.merge(disgenet, gen_name_gen_id, left_on='geneSymbol', right_on='gene_name', how='inner')
disgenet_join = disgenet_join.drop_duplicates()
disgenet_result = disgenet_join.drop(columns=["geneSymbol", "gene_name", "geneId"])
df_col = disgenet_result.pop("gene_id")
disgenet_result.insert(0, "gene_id", df_col)
disgenet_result.to_csv("./data/disgenet_mapped.csv", sep="\t", index=False)
disgenet_result
#miért lett több eleme az inner join után mint volt a sima disgenetnek ?

,gene_id,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,ENSG00000121410,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.000,2017.0,2017.0,1,0,CTD_human
1,ENSG00000121410,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.000,2015.0,2015.0,1,0,CTD_human
2,ENSG00000175899,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
3,ENSG00000175899,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.000,2004.0,2019.0,1,0,CTD_human
4,ENSG00000175899,0.529,0.769,C0009375,Colonic Neoplasms,group,C06;C04,Neoplastic Process,0.30,1.000,2004.0,2004.0,1,0,CTD_human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92954,ENSG00000251908,0.931,0.077,C4722327,"PROSTATE CANCER, HEREDITARY, 1",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human
92955,ENSG00000222259,0.931,0.077,C2931456,"Prostate cancer, familial",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human
92956,ENSG00000222259,0.931,0.077,C4722327,"PROSTATE CANCER, HEREDITARY, 1",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human
92957,ENSG00000207222,0.931,0.077,C2931456,"Prostate cancer, familial",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human


A gtex géneket tartalmaz a fehérje leírók pedig fehérje azonosítókat, ezért az elejéről le kell vágni a betüket.

In [8]:
gtex = pd.read_csv("./data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct", sep='\t', header=0)
gtex_sorted = gtex.sort_values(by="Name", ascending=True)
gtex_sorted["Name"] = gtex_sorted.apply(lambda row: row["Name"][4:].split(".")[0], axis=1)
gtex_sorted= gtex_sorted.reset_index(drop=True)
#jól gondolom, hogy a fehérje és gén azonosító az ugyan arra mutat ha el távolitom az elejéről a ENSG illetve ENSP stringeket?

A string adathalmazból a legerősebb fehérje fehérje interakcióra van szükségünk. Ezek csak egyszer szerepeljenek a listában.

In [9]:
string = pd.read_csv("./data/string_9606.protein.links.v12.0.txt", sep=' ', header=0)
string["protein1"] = string["protein1"].str.slice(5)
string["protein2"] = string["protein2"].str.slice(5)
string["combined_score"] = string["combined_score"] / 1000
string.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13715404 entries, 0 to 13715403
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   protein1        object 
 1   protein2        object 
 2   combined_score  float64
dtypes: float64(1), object(2)
memory usage: 313.9+ MB


In [10]:
string_sorted = string.sort_values(by=["protein1", "combined_score"], ascending=[False, False])
string_filtered = string_sorted.drop_duplicates(subset=['protein1'], keep='first')
string_filtered = string_filtered.reset_index(drop=True)
string_filtered["tuple"] = string_filtered.apply(lambda row: tuple(sorted([row['protein1'], row['protein2']])), axis=1)
string_filtered = string_filtered.drop_duplicates(subset=['tuple'])
string_filtered = string_filtered.drop("tuple", axis=1)
string_filtered.to_csv("./data/string_filtered.csv", index=False, sep="\t")

Azon gének megtalálása melyek szerepelnek a stringben és a gtex leíróban is

In [11]:
string_sorted = string_filtered
string_sorted["protein1"] = string_sorted.apply(lambda row: row["protein1"][4:], axis=1)
string_sorted["protein2"] = string_sorted.apply(lambda row: row["protein2"][4:], axis=1)
gtex_mapped_p1 = pd.merge(gtex_sorted, string_sorted, left_on='Name', right_on='protein1', how='inner')
gtex_mapped_p2 = pd.merge(gtex_sorted, string_sorted, left_on='Name', right_on='protein2', how='inner')
gtex_mapped = pd.concat([gtex_mapped_p1, gtex_mapped_p2], ignore_index=True)
gtex_mapped_result = gtex_mapped.drop(columns=["protein1", "protein2", "combined_score"])
gtex_mapped_result = gtex_mapped_result.drop_duplicates()
gtex_mapped_result.to_csv("./data/gtex_mapped_genes.csv", sep="\t", index=False)